In [2]:
import fitz # pymupdf
import re

I am not sure comparing the TOC is useful but I'm leaving this in, just in case there are issues later

In [84]:
header_size=70
footer_size=80


def create_toc_as_list(file_path, header_size=70, footer_size=80):
    doc = fitz.open(file_path)
    toc_as_list = []
    toc_start = False
    end_of_toc_found = False

    for page in doc:
        tl = page.rect[0], page.rect[1]  # lower-left coordinates
        br = page.rect[2], page.rect[3]  # upper-right
        rect = fitz.Rect(tl[0], tl[1]+header_size, br[0], br[1]-footer_size)
        lines = page.get_text('text', clip=rect).split('\n')

        for line in lines:
            if 'Table of Contents' in line:
                toc_start = True
                continue
            if not toc_start:
                continue
            if line.strip() == "Legal context":
                end_of_toc_found = True
                break
            toc_as_list.append(line)

        if end_of_toc_found:
            break
    return toc_as_list


In [31]:
pdf_file = "./source/Currency and Exchanges Manual for Authorised Dealers_20230523.pdf"
base_doc = fitz.open(pdf_file)
#base_toc_list = create_toc_as_list(pdf_file)
pdf_file = "./source/Currency and Exchanges Manual for Authorised Dealers_20231013.pdf"
new_doc = fitz.open(pdf_file)
#new_toc_list = create_toc_as_list(pdf_file)

In [67]:
header_size=70
footer_size=80

def get_page_text_as_list(pymupdf_doc, page_number):
    page = pymupdf_doc[page_number]
    tl = page.rect[0], page.rect[1]  # lower-left coordinates
    br = page.rect[2], page.rect[3]  # upper-right
    rect = fitz.Rect(tl[0], tl[1]+header_size, br[0], br[1]-footer_size)
    lines = page.get_text('text', clip=rect).split('\n')
    lines = [s.strip() for s in lines]

    # Remove empty strings from the start
    while lines and not lines[0]:
        lines.pop(0)

    # Remove empty strings from the end
    while lines and not lines[-1]:
        lines.pop()

    return lines

In [85]:
class PageInfo:
    def __init__(self, start, end, offset):
        if end <= start:
            raise ValueError("End must be greater than Start.")
        self.start = start
        self.end = end
        self.offset = offset

def create_page_list(raw_list):
    sorted_list = sorted(raw_list, key=lambda x: x['start'])
    page_list = []
    
    for i, item in enumerate(sorted_list):
        start = item['start']
        end = item['end']
        offset = item['offset']
        
        if i > 0 and start != page_list[-1].end:
            raise ValueError("The end of an item must be the start of the next item.")

        page_list.append(PageInfo(start, end, offset))
    
    return page_list

def find_offset(page_list, i):
    for item in page_list:
        if item.start <= i < item.end:
            return item.offset



# i = 15
# offset = find_offset(page_list, i)
# print(offset)

0


In [88]:
find_offset(page_list, 21)

1

In [89]:
raw_list = [
    {'start': 13, 'end': 22, 'offset': 0},
    {'start': 22, 'end': 1000, 'offset': 1}
]
page_list = create_page_list(raw_list)

# The most common update is to a table of Authorised Dealers. 
try:
    for i in range(13, len(base_doc)):
        page_offset = find_offset(page_list, i)
        base_lines = get_page_text_as_list(base_doc, i)
        new_lines = get_page_text_as_list(new_doc, i + page_offset)

        if len(new_lines) != len(base_lines):
            print(f"Page {i} line lengths dont match")
            break
        else:
            for j in range(len(new_lines)):
                if new_lines[j].strip() != base_lines[j].strip():
                    print(f"Page {i}, line {j} does not match")
                    break
        if i % 50 == 0:
            print(f"{i} lines processed")
except IndexError as e:
    print(f"error on page {i}")



Page 21 line lengths dont match


In [82]:
10 % 50


10

In [62]:
bl = get_page_text_as_list(base_doc, 27)[0]
nl = get_page_text_as_list(new_doc, 28)[0]

In [64]:
nl

'29 of 294'

In [38]:
len(new_lines) == len(base_lines)

True

In [30]:
base_toc_list

['Legal context ........................................................................................................................ 13 ',
 'Introduction .......................................................................................................................... 14 ',
 'A.1 ',
 'Definitions ............................................................................................................... 15 ',
 'A.2 ',
 'Authorised entities ................................................................................................. 21 ',
 '(A) ',
 'Authorised Dealers ................................................................................................... 21 ',
 '(B) ',
 'Authorised Dealers in foreign exchange with limited authority .................................. 22 ',
 'A.3 ',
 'Duties and responsibilities of Authorised Dealers .............................................. 24 ',
 '(A) ',
 'Introduction .................................................

In [14]:
for i in range(len(base_toc_list)):
    if base_toc_list[i].strip().replace(".","") != new_toc_list[i].strip().replace(".",""):
        print(f'Mismatch at line {i}')

Mismatch at line 11
Mismatch at line 13
Mismatch at line 16
Mismatch at line 18
Mismatch at line 20
Mismatch at line 22
Mismatch at line 25
Mismatch at line 27
Mismatch at line 29
Mismatch at line 32
Mismatch at line 34
Mismatch at line 36
Mismatch at line 38
Mismatch at line 40
Mismatch at line 42
Mismatch at line 44
Mismatch at line 46
Mismatch at line 48
Mismatch at line 50
Mismatch at line 52
Mismatch at line 54
Mismatch at line 56
Mismatch at line 58
Mismatch at line 60
Mismatch at line 62
Mismatch at line 65
Mismatch at line 67
Mismatch at line 70
Mismatch at line 72
Mismatch at line 74
Mismatch at line 76
Mismatch at line 78
Mismatch at line 80
Mismatch at line 83
Mismatch at line 85
Mismatch at line 87
Mismatch at line 90
Mismatch at line 92
Mismatch at line 94
Mismatch at line 96
Mismatch at line 98
Mismatch at line 101
Mismatch at line 103
Mismatch at line 105
Mismatch at line 107
Mismatch at line 109
Mismatch at line 111
Mismatch at line 113
Mismatch at line 116
Mismatch at 

In [18]:
i = 13
print(base_toc_list[-1].strip().replace(".",""))
print(new_toc_list[-1].strip().replace(".",""))